In [7]:
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

from IPython.display import display

from sklearn import svm
from sklearn import metrics
from sklearn import ensemble
from sklearn import neighbors
from sklearn import feature_selection

In [6]:
# Naive Bayes: Apenas um experimento para servir de baseline
# Decision Tree: Variar a altura máxima da árvore (incluindo permitir altura ilimitada) e 
# mostrar os resultados graficamente
# SVM: Avaliar os kernels linear, sigmoid, polinomial e RBF
# k-NN: Variar o número k de vizinhos e mostrar os resultados graficamente
# Random Forest: Variar o número de árvores e mostrar os resultados graficamente.
# Gradient Tree Boosting: Variar o número de iterações e mostrar os resultados graficamente. 

In [32]:
#Input filepath 
INPUT_FILEPATH = "koi_data.csv"
TARGET = "koi_disposition"
N_FEATURES = 40

df = pd.read_csv(INPUT_FILEPATH)
df = df.drop(["kepoi_name"], axis=1)

print("lines: {}".format(df.shape[0]))
print("rows: {}".format(df.shape[1]))
print("Missing data: {}".format(df.isnull().sum().sum()))


print("\n InputFile:")
with pd.option_context("max_columns", 40): # Limita o numero de cols mostradas
    display(df.head(20))

# list features
features = list(df.columns)
features.remove(TARGET)
print("Target: {}".format(TARGET))

# print("Features:")
# print("\n".join(["  " + x for x in features]))


lines: 5202
rows: 42
Missing data: 0

 InputFile:


,koi_disposition,koi_period,koi_impact,koi_duration,koi_depth,koi_ror,koi_srho,koi_prad,koi_sma,koi_incl,koi_teq,koi_insol,koi_dor,koi_max_sngle_ev,koi_max_mult_ev,koi_model_snr,koi_steff,koi_slogg,koi_smet,koi_srad,...,koi_gmag,koi_rmag,koi_imag,koi_zmag,koi_jmag,koi_hmag,koi_kmag,koi_fwm_stat_sig,koi_fwm_sra,koi_fwm_sdec,koi_fwm_srao,koi_fwm_sdeco,koi_fwm_prao,koi_fwm_pdeco,koi_dicco_mra,koi_dicco_mdec,koi_dicco_msky,koi_dikco_mra,koi_dikco_mdec,koi_dikco_msky
0,CONFIRMED,9.48804,0.146,2.95750,615.8,0.02234,3.20796,2.26,0.0853,89.66,793.0,93.59,24.8100,5.13585,28.47082,35.8,5455.0,4.467,0.14,0.927,...,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.002,19.46229,48.14191,0.4300,0.9400,-0.00020,-0.00055,-0.010,0.200,0.200,0.080,0.310,0.320
1,CONFIRMED,54.41838,0.586,4.50700,874.8,0.02795,3.02368,2.83,0.2734,89.57,443.0,9.11,77.9000,7.02767,20.10951,25.8,5455.0,4.467,0.14,0.927,...,15.890,15.270,15.114,15.006,14.082,13.751,13.648,0.003,19.46226,48.14199,-0.6300,1.2300,0.00066,-0.00105,0.390,0.000,0.390,0.490,0.120,0.500
2,FALSE POSITIVE,1.73695,1.276,2.40641,8079.2,0.38739,0.22080,33.46,0.0267,67.09,1395.0,891.96,3.2780,39.06655,541.89510,505.6,5805.0,4.564,-0.52,0.791,...,16.100,15.554,15.382,15.266,14.326,13.911,13.809,0.000,19.03564,48.28521,-0.1110,0.0020,0.00302,-0.00142,-0.249,0.147,0.289,-0.257,0.099,0.276
3,CONFIRMED,2.52559,0.701,1.65450,603.3,0.02406,1.98635,2.75,0.0374,85.41,1406.0,926.16,8.7500,4.74994,33.19190,40.9,6031.0,4.438,0.07,1.046,...,16.015,15.468,15.292,15.241,14.366,14.064,13.952,0.733,19.25033,48.22626,-0.0100,0.2300,0.00008,-0.00007,0.030,-0.090,0.100,0.070,0.020,0.070
4,FALSE POSITIVE,7.36179,1.169,5.02200,233.7,0.18339,0.00485,39.21,0.0820,60.92,1342.0,767.22,2.4000,10.96468,46.15308,47.7,6227.0,3.986,0.00,1.958,...,12.999,12.609,12.515,12.480,11.659,11.415,11.396,0.000,19.92395,42.15826,-13.4500,24.0900,0.00303,-0.00555,-4.506,7.710,8.930,-4.537,7.713,8.948
5,CONFIRMED,16.06865,0.052,3.53470,4914.3,0.06216,3.66590,5.76,0.1158,89.92,600.0,30.75,36.8500,20.45716,137.44981,161.9,5031.0,4.485,0.16,0.848,...,16.559,15.770,15.534,15.368,14.363,13.868,13.785,0.063,19.13330,48.37578,0.0330,-0.0440,0.00026,0.00049,0.041,-0.015,0.044,0.005,0.030,0.031
6,CONFIRMED,2.47061,0.818,1.74319,14231.0,0.12387,1.83538,13.04,0.0354,84.41,1339.0,761.46,8.3977,544.17010,6468.04000,4304.3,5820.0,4.457,-0.06,0.964,...,11.736,11.275,11.168,11.126,10.232,9.920,9.846,0.000,19.12056,49.31643,-0.0232,0.1066,-0.00012,0.00078,-0.009,-0.300,0.300,0.015,-0.270,0.270
7,CONFIRMED,3.52250,0.631,3.19843,9145.7,0.09209,0.65374,14.59,0.0473,85.20,1521.0,1264.67,7.5410,130.83832,1725.58240,1741.5,6225.0,4.169,-0.04,1.451,...,13.886,13.511,13.424,13.413,12.576,12.324,12.293,0.000,18.75253,42.45106,-0.5515,-0.0597,0.00001,0.00036,0.014,-0.013,0.020,-0.028,0.176,0.178
8,CONFIRMED,3.70921,0.051,2.63020,131.1,0.01042,1.77451,1.16,0.0465,89.73,1206.0,500.46,10.8880,4.86573,41.28873,50.6,5833.0,4.407,-0.04,1.022,...,13.169,12.740,12.602,12.564,11.698,11.402,11.367,0.009,19.70992,48.49568,0.3600,0.4500,-0.00005,-0.00005,0.240,-0.150,0.290,0.160,-0.130,0.210
9,FALSE POSITIVE,11.52145,2.483,3.63990,17984.0,1.62536,1.27376,150.51,0.0978,83.13,753.0,75.88,20.7500,92.90157,628.37080,622.1,5795.0,4.554,-0.20,0.848,...,15.994,15.419,15.249,15.153,14.248,13.831,13.827,0.895,19.80533,47.59739,0.0680,-0.0380,0.00000,0.00010,0.009,-0.009,0.013,0.147,0.037,0.151


Target: koi_disposition
